In [1]:
from cvzone.HandTrackingModule import HandDetector
import cv2
import cvzone
import numpy as np

In [3]:
cap = cv2.VideoCapture(0)
cap.set(3,1288) #width
cap.set(4,720) #height
detector = HandDetector(detectionCon = 0.8)

#width and height and center of rectangle
cx_exit, cy_exit, w_exit, h_exit = 1000, 1000, 100, 100 #exit

class DragRec():
    def __init__(self,posCenter, size=[200,200], color=(255, 0, 255)):
        self.posCenter = posCenter
        self.size = size
        self.color = color
    
    def updateRec_color(self):
        self.color = (255, 0, 255)
    
    def updateRec(self,cursor):
        cx,cy = self.posCenter
        w,h = self.size
        
        # check if index tip is in rectangle
        if cx-w//2< cursor[0]< cx+w//2 and cy-h//2<cursor[1]<cy+h//2: 
            self.posCenter = cursor #rectangle follow tip
            self.color = (0,255,0)
            

# create rect
rectList = []
for i in range (3):
    rectList.append(DragRec([i*250+150,150])) #not in same place

# main function
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1) #mirror
    img = detector.findHands(img)
    lmList, _ = detector.findPosition(img)

    if lmList:
        #find distance between finger(got click onot)
        l, _, _ = detector.findDistance(8,12,img, draw=False) #8=index tip, 12=middle tip
#         print(l)
        if l<50:   
            cursor = lmList[8] #x and y of tip of index finger
            # call updateRec
            for rect in rectList:
                rect.updateRec(cursor)
        else:
            for rect in rectList:
                rect.updateRec_color()

    ### get latest rect position and draw
    ## Draw Solid
#     for rect in rectList:
#         cx,cy = rect.posCenter
#         w,h = rect.size
#         color = rect.color
#         cv2.rectangle(img, (cx-w//2,cy-h//2), (cx+w//2,cy+h//2), color, cv2.FILLED)
#         cvzone.cornerRect(img, (cx-w//2, cy-h//2, w, h),20,rt=0)
#     cv2.imshow("Image",img)

    ## Draw Transparent
    imgNew = np.zeros_like(img,np.uint8)
    for rect in rectList:
        cx,cy = rect.posCenter
        w,h = rect.size
        color = rect.color
        cv2.rectangle(imgNew, (cx-w//2,cy-h//2), (cx+w//2,cy+h//2), color, cv2.FILLED)
        cvzone.cornerRect(imgNew, (cx-w//2, cy-h//2, w, h),20,rt=0)
    out = img.copy()
    alpha = 0.2
    mask = imgNew.astype(bool)
    out[mask] = cv2.addWeighted(img, alpha, imgNew, 1-alpha, 0)[mask]
    cv2.imshow("Image",out)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
    
cap.release()
cv2.destroyAllWindows()